<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza y Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano y Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Simon Repolt
- Magdalena de la Fuente


### **Link de repositorio de GitHub:** `https://github.com/HexaPulsar/LabMDS`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC,SVR,LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 24.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones.

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.csv`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
"""# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')"""

'# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.\ntry:\n    from google.colab import drive\n    drive.mount("/content/drive")\n    path = \'Dirección donde tiene los archivos en el Drive\'\nexcept: \n    print(\'Ignorando conexión drive-colab\')'

In [3]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego.

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país.

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
### Código aquí ###
df_players['National_Position'] = df_players['National_Position'].apply(lambda x: 0 if isinstance(x, float) else 1)

gb_df_players = df_players.groupby(by = "National_Position").agg('mean').round(2)
gb_df_players['Count'] = df_players['National_Position'].value_counts()
display(gb_df_players)

<ipython-input-4-d8fbff2cb392>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gb_df_players = df_players.groupby(by = "National_Position").agg('mean').round(2)


,Height,Weight,Age,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Count
National_Position,,,,,,,,,,,,,,,,,,,,,
0,181.04,75.15,25.37,2.92,2.29,57.57,54.43,43.93,45.31,47.13,...,64.72,52.09,55.16,44.77,46.95,46.79,43.00,48.87,42.85,16513
1,182.05,76.79,26.91,3.13,2.47,64.11,60.51,48.77,49.43,52.17,...,68.02,57.03,62.11,51.13,54.41,53.19,49.27,53.77,49.81,1075


In [5]:
df_players.describe().round(2)

,National_Position,Height,Weight,Age,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
count,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00,...,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00,17588.00
mean,0.06,181.11,75.25,25.46,2.93,2.30,57.97,54.80,44.23,45.57,...,63.21,64.92,52.39,55.58,45.16,47.40,47.18,43.38,49.17,43.28
std,0.24,6.68,6.90,4.68,0.66,0.75,16.83,18.91,21.56,21.52,...,14.62,11.43,17.47,17.60,19.37,19.21,18.46,17.70,15.87,17.71
min,0.00,155.00,48.00,17.00,1.00,1.00,5.00,4.00,3.00,5.00,...,11.00,15.00,4.00,3.00,2.00,4.00,6.00,4.00,7.00,3.00
25%,0.00,176.00,70.00,22.00,3.00,2.00,53.00,47.00,22.00,23.00,...,55.00,58.00,45.00,45.00,29.00,32.00,34.00,31.00,39.00,30.00
50%,0.00,181.00,75.00,25.00,3.00,2.00,63.00,60.00,48.00,51.00,...,65.00,65.00,56.00,59.00,48.00,52.00,48.00,42.00,50.00,44.00
75%,0.00,186.00,80.00,29.00,3.00,3.00,69.00,68.00,64.00,64.00,...,74.00,73.00,65.00,69.00,61.00,63.00,62.00,57.00,61.00,57.00
max,1.00,207.00,110.00,47.00,5.00,5.00,95.00,97.00,92.00,95.00,...,96.00,95.00,94.00,93.00,95.00,91.00,92.00,93.00,96.00,93.00


### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores.

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [6]:
X = df_players.drop('National_Position', axis=1)
y = df_players['National_Position']
categorical = X.select_dtypes(include=['object']).columns  # or exclude=['number'] for older versions
numerical = X.select_dtypes(include=['number']).columns  # or include=['number'] for older versions



In [7]:
transformers = [
    ('num', MinMaxScaler(), numerical),
    ('cat', OneHotEncoder(handle_unknown='ignore'),categorical)
]

# Create the column transformer
preprocessor = ColumnTransformer(transformers)


tree_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier',DecisionTreeClassifier())
])


l_svc_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LinearSVC())
])

rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])


Justificación de preprocesamiento:

Se escogen dos formas de preprocesar el dataset: un procesador para las variables categoricas y otro para las numericas. Para las numericas se utiliza un minmaxscaler debido a que esto va a cambiar las variables y dejarlas en un rango definido entre -1 y 1, lo cual ayuda mucho a los modelos en generar clasificaciones y en procesar los datos ya que al escalarlos se elimina variabilidad entre ellos y tambien las diferencias de escala y magnitud que hay entre los datos.

Por otra parte, se utiliza one hot encoder para las categoricas ya que de esta forma pasan a ser variables numericas que son 0 o 1, permitiendo tambien procesar valores de texto

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
def train(X_train, X_test, y_train, y_test,pipeline):



    pipeline.fit(X_train, y_train)
    display(pipeline)
    # Make predictions using the fitted pipeline
    y_pred = pipeline.predict(X_test)
    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    F1 = f1_score(y_test,y_pred,average='macro')
    print(f'Accuracy: {accuracy:.2f}')
    print(f'F1: {F1:.2f}')
    print(f'-'*60)
    print(classification_report(y_test, y_pred))



train(X_train, X_test, y_train, y_test,l_svc_pipeline)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', LinearSVC())])

Accuracy: 0.95
F1: 0.71
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      3324
           1       0.66      0.34      0.45       194

    accuracy                           0.95      3518
   macro avg       0.81      0.66      0.71      3518
weighted avg       0.95      0.95      0.95      3518



In [9]:
train(X_train, X_test, y_train, y_test,tree_pipeline)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', DecisionTreeClassifier())])

Accuracy: 0.93
F1: 0.66
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3324
           1       0.38      0.34      0.36       194

    accuracy                           0.93      3518
   macro avg       0.67      0.65      0.66      3518
weighted avg       0.93      0.93      0.93      3518



In [10]:
train(X_train, X_test, y_train, y_test,rf_pipeline)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

Accuracy: 0.95
F1: 0.54
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      3324
           1       0.79      0.06      0.11       194

    accuracy                           0.95      3518
   macro avg       0.87      0.53      0.54      3518
weighted avg       0.94      0.95      0.92      3518



**Respuesta**:

Volviendo a la distribución de la data observada en `gb_df_players`

In [11]:
display(gb_df_players)

,Height,Weight,Age,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Count
National_Position,,,,,,,,,,,,,,,,,,,,,
0,181.04,75.15,25.37,2.92,2.29,57.57,54.43,43.93,45.31,47.13,...,64.72,52.09,55.16,44.77,46.95,46.79,43.00,48.87,42.85,16513
1,182.05,76.79,26.91,3.13,2.47,64.11,60.51,48.77,49.43,52.17,...,68.02,57.03,62.11,51.13,54.41,53.19,49.27,53.77,49.81,1075


Se observa un severo desbalance de clases (columna Count). los desbalances de clases tan severos pueden introducir un bias en el modelo, confundiendo al modelo. En general, este bias se reduce haciendo un balanceo de clases antes de entrenar. Para esto, podemos modificar el pipeline de preprocesamiento descrito más arriba o bien utilizar una técnica de stratificación, en donde los subdatasets construidos (X_train,X_test,y_train,y_test) se balancean durante el split. Debido a esto, importa mucho la cantidad de elementos por clase que hay en el dataset.

Por otra parte, para la clase positiva se pueden notar recalls, precisions y f1 scores  muy bajos, en contraste con la clase negativa. Lo anterior se produce por el desbalance de clases tal como se menciono anteriormente, y esto esta generando que, al modelo le esta costando predecir la clase positiva (precision) y a la vez le esta costando encontrar todos los objetos pertenecientes a la clase positiva (recall)



In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


train(X_train, X_test, y_train, y_test,l_svc_pipeline)
train(X_train, X_test, y_train, y_test,tree_pipeline)
train(X_train, X_test, y_train, y_test,rf_pipeline)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', LinearSVC())])

Accuracy: 0.95
F1: 0.73
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      3303
           1       0.69      0.37      0.48       215

    accuracy                           0.95      3518
   macro avg       0.82      0.68      0.73      3518
weighted avg       0.94      0.95      0.94      3518



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', DecisionTreeClassifier())])

Accuracy: 0.93
F1: 0.68
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3303
           1       0.44      0.35      0.39       215

    accuracy                           0.93      3518
   macro avg       0.70      0.66      0.68      3518
weighted avg       0.93      0.93      0.93      3518



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

Accuracy: 0.94
F1: 0.52
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       0.75      0.04      0.08       215

    accuracy                           0.94      3518
   macro avg       0.85      0.52      0.52      3518
weighted avg       0.93      0.94      0.91      3518



In [13]:
!pip install imblearn
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)


train(X_train_resampled, X_test, y_train_resampled, y_test,l_svc_pipeline)
train(X_train_resampled, X_test, y_train_resampled, y_test,tree_pipeline)
train(X_train, X_test, y_train, y_test,rf_pipeline)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', LinearSVC())])

Accuracy: 0.95
F1: 0.76
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      3303
           1       0.65      0.48      0.55       215

    accuracy                           0.95      3518
   macro avg       0.81      0.73      0.76      3518
weighted avg       0.95      0.95      0.95      3518



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', DecisionTreeClassifier())])

Accuracy: 0.91
F1: 0.69
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.94      0.95      3303
           1       0.36      0.52      0.43       215

    accuracy                           0.91      3518
   macro avg       0.66      0.73      0.69      3518
weighted avg       0.93      0.91      0.92      3518



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

Accuracy: 0.94
F1: 0.52
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       0.75      0.04      0.08       215

    accuracy                           0.94      3518
   macro avg       0.85      0.52      0.52      3518
weighted avg       0.93      0.94      0.91      3518



## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas.

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características.

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [14]:
### Código aquí ###d
df_players['Club_Position']
cond = df_players.query("Club_Position != 'Res' and Club_Position != 'Sub'")
display(cond)

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,1,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,1,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,1,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,1,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,1,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17310,Tom Bolarinwa,England,0,RM,180,80,Right,26,Medium / Medium,3,...,54,54,22,38,37,37,37,32,29,29
17312,Zak Mills,England,0,RB,182,83,Right,24,Medium / Medium,4,...,58,61,43,22,24,28,30,32,39,31
17442,Wataru Hashimoto,Japan,0,LB,181,72,Left,30,Medium / High,2,...,64,71,41,29,26,26,25,25,31,29
17461,Callum Cooke,England,0,LCM,172,72,Right,20,Medium / Medium,3,...,60,63,57,48,50,54,42,33,69,40


In [125]:
gb_cond = cond.groupby('Club_Position').agg('mean').round(2)
display(gb_cond)

n_cond = cond.groupby('Club_Position').size().reset_index(name='Count')
display(n_cond)

<ipython-input-125-1690ece3c170>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gb_cond = cond.groupby('Club_Position').agg('mean').round(2)


,National_Position,Height,Weight,Age,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
Club_Position,,,,,,,,,,,,,,,,,,,,,
CAM,0.10,177.23,71.94,26.89,3.32,3.03,71.60,71.00,38.19,39.97,...,74.45,62.91,53.50,68.41,62.93,65.87,65.97,63.89,62.76,59.87
CB,0.07,186.43,80.86,28.17,2.77,2.02,55.30,44.52,68.51,67.01,...,52.48,69.59,69.25,54.29,33.99,40.17,39.19,39.23,43.47,33.35
CDM,0.10,181.21,75.24,27.97,3.03,2.27,68.87,63.06,64.01,64.34,...,64.02,66.19,60.18,65.52,48.86,59.02,55.93,52.94,54.81,49.17
CF,0.25,179.25,76.00,26.75,3.50,2.50,66.75,67.75,29.75,34.50,...,71.25,65.25,55.25,64.75,66.00,63.25,60.25,58.75,66.75,66.00
CM,0.09,178.81,72.94,27.04,3.09,2.59,69.72,66.44,53.99,56.27,...,69.43,62.89,55.95,66.77,54.05,62.94,62.76,59.84,59.32,54.58
GK,0.13,188.41,83.01,28.82,2.57,1.00,20.98,14.74,13.55,14.58,...,44.16,61.84,14.70,22.87,13.36,14.45,15.21,15.46,22.05,14.17
LAM,0.08,175.17,71.03,27.50,3.39,3.53,71.39,72.11,35.28,35.58,...,77.78,58.22,51.64,64.06,64.83,64.89,64.97,61.31,62.00,60.17
LB,0.07,178.74,73.50,26.45,2.81,2.24,64.24,62.80,64.97,67.11,...,69.31,68.73,57.00,58.49,41.44,50.03,56.24,49.08,48.22,41.59
LCB,0.08,186.01,80.09,27.21,2.82,2.02,55.79,45.61,68.27,67.88,...,53.65,70.23,68.18,52.71,32.08,37.70,38.86,37.84,42.80,34.01


,Club_Position,Count
0,CAM,315
1,CB,83
2,CDM,119
3,CF,4
4,CM,79
5,GK,632
6,LAM,36
7,LB,549
8,LCB,631
9,LCM,354


In [16]:
cond = cond.dropna()
X = cond.drop('Club_Position', axis=1)
y = cond['Club_Position']
categorical = X.select_dtypes(include=['object']).columns  # or exclude=['number'] for older versions
numerical = X.select_dtypes(include=['number']).columns  # or include=['number'] for older versions

transformers = [
    ('num', MinMaxScaler(), numerical),
    ('cat', OneHotEncoder(handle_unknown='ignore'),categorical)
]

# Create the column transformer
preprocessor = ColumnTransformer(transformers)


svc_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])

tree_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier',DecisionTreeClassifier())
])
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train(X_train, X_test, y_train, y_test,svc_pipeline)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['National_Position', 'Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', SVC())])

Accuracy: 0.46
F1: 0.22
------------------------------------------------------------
              precision    recall  f1-score   support

         CAM       0.27      0.31      0.29        64
          CB       0.00      0.00      0.00        16
         CDM       0.00      0.00      0.00        27
          CF       0.00      0.00      0.00         1
          CM       0.00      0.00      0.00        21
          GK       1.00      1.00      1.00       112
         LAM       0.00      0.00      0.00         7
          LB       0.55      0.78      0.65        93
         LCB       0.71      0.34      0.46       135
         LCM       0.16      0.10      0.13        67
         LDM       0.16      0.12      0.14        48
          LF       0.00      0.00      0.00         2
          LM       0.27      0.24      0.25        99
          LS       0.47      0.17      0.25        40
          LW       0.00      0.00      0.00        28
         LWB       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
train(X_train, X_test, y_train, y_test,tree_pipeline)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['National_Position', 'Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', DecisionTreeClassifier())])

Accuracy: 0.35
F1: 0.20
------------------------------------------------------------
              precision    recall  f1-score   support

         CAM       0.15      0.17      0.16        64
          CB       0.00      0.00      0.00        16
         CDM       0.12      0.11      0.12        27
          CF       0.00      0.00      0.00         1
          CM       0.11      0.10      0.10        21
          GK       1.00      1.00      1.00       112
         LAM       0.00      0.00      0.00         7
          LB       0.62      0.60      0.61        93
         LCB       0.42      0.44      0.43       135
         LCM       0.15      0.19      0.17        67
         LDM       0.15      0.17      0.16        48
          LF       0.00      0.00      0.00         2
          LM       0.23      0.16      0.19        99
          LS       0.18      0.20      0.19        40
          LW       0.05      0.04      0.04        28
         LWB       0.00      0.00      0.00       

In [19]:
train(X_train, X_test, y_train, y_test,rf_pipeline)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['National_Position', 'Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

Accuracy: 0.44
F1: 0.21
------------------------------------------------------------
              precision    recall  f1-score   support

         CAM       0.22      0.25      0.23        64
          CB       0.00      0.00      0.00        16
         CDM       1.00      0.04      0.07        27
          CF       0.00      0.00      0.00         1
          CM       0.00      0.00      0.00        21
          GK       1.00      1.00      1.00       112
         LAM       0.00      0.00      0.00         7
          LB       0.52      0.75      0.61        93
         LCB       0.51      0.44      0.47       135
         LCM       0.12      0.13      0.12        67
         LDM       0.20      0.10      0.14        48
          LF       0.00      0.00      0.00         2
          LM       0.23      0.25      0.24        99
          LS       0.40      0.05      0.09        40
          LW       0.50      0.07      0.12        28
         LWB       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
#anadi stratify
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


train(X_train, X_test, y_train, y_test,svc_pipeline)
train(X_train, X_test, y_train, y_test,tree_pipeline)
train(X_train, X_test, y_train, y_test,rf_pipeline)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['National_Position', 'Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', SVC())])

Accuracy: 0.46
F1: 0.20
------------------------------------------------------------
              precision    recall  f1-score   support

         CAM       0.29      0.27      0.28        63
          CB       0.00      0.00      0.00        16
         CDM       0.00      0.00      0.00        24
          CF       0.00      0.00      0.00         1
          CM       0.00      0.00      0.00        16
          GK       1.00      1.00      1.00       126
         LAM       0.00      0.00      0.00         7
          LB       0.55      0.75      0.64       110
         LCB       0.59      0.21      0.31       126
         LCM       0.16      0.06      0.08        71
         LDM       0.28      0.19      0.22        53
          LF       0.00      0.00      0.00         2
          LM       0.25      0.23      0.24        83
          LS       0.33      0.15      0.20        41
          LW       0.00      0.00      0.00        27
         LWB       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['National_Position', 'Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', DecisionTreeClassifier())])

Accuracy: 0.36
F1: 0.21
------------------------------------------------------------
              precision    recall  f1-score   support

         CAM       0.21      0.21      0.21        63
          CB       0.11      0.12      0.11        16
         CDM       0.10      0.08      0.09        24
          CF       0.00      0.00      0.00         1
          CM       0.00      0.00      0.00        16
          GK       1.00      1.00      1.00       126
         LAM       0.00      0.00      0.00         7
          LB       0.53      0.55      0.54       110
         LCB       0.39      0.37      0.38       126
         LCM       0.12      0.14      0.13        71
         LDM       0.02      0.02      0.02        53
          LF       0.00      0.00      0.00         2
          LM       0.30      0.25      0.27        83
          LS       0.22      0.24      0.23        41
          LW       0.11      0.15      0.12        27
         LWB       0.07      0.11      0.08       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  Index(['National_Position', 'Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

Accuracy: 0.45
F1: 0.22
------------------------------------------------------------
              precision    recall  f1-score   support

         CAM       0.24      0.29      0.26        63
          CB       0.00      0.00      0.00        16
         CDM       0.00      0.00      0.00        24
          CF       0.00      0.00      0.00         1
          CM       0.00      0.00      0.00        16
          GK       1.00      1.00      1.00       126
         LAM       0.00      0.00      0.00         7
          LB       0.57      0.74      0.64       110
         LCB       0.44      0.39      0.41       126
         LCM       0.18      0.17      0.17        71
         LDM       0.25      0.15      0.19        53
          LF       0.00      0.00      0.00         2
          LM       0.23      0.29      0.26        83
          LS       0.71      0.12      0.21        41
          LW       0.38      0.11      0.17        27
         LWB       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Respuesta**:


De los resultados se puede observar, que para todos los modelos, las métricas obtenidas son consistentemente malas en todas las prosiciones. Cabe decir que hay algunas posiciones donde hay mejores predicciones que en las otras, sin embargo, la norma general es que hayan malos resultados. Esto puede ser debido a que la cantidad de clases es muy grande, complejizando mucho mas el problema de clasificacion, además de que las clases estan desbalanceadas.

Una buena solucion seria reducir la cantidad de clases a clasificar, agrupando las posiciones de acuerdo a su localizacion en la cancha (ej: delanteros, mediocampo, defensas) y a esas 3 grandes clases, asignar subclases, de esta forma habria que predecir 3 clases y no mas de 10 de ellas, además que de al hacer esto se estarian balanceando los datasets.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [X] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

La métrica R^2 es una forma de cuantificar que tan lejos los datos se encuentran de la linea de regresión que fue ajustada, en otras palabras, representa de cierta forma error entre el ajuste y los datos. La métrica en particular calcula un error con respecto a la esperanza de los valores de la variable a estimar. Si la estimación tiene 0 error con respecto a dicha esperanza, el R^2 es 1 (o 0, dependiendo de como se considere el calculo). Por otra parte, si la estimación no logra fittear correctamente ningun dato, tendrá un R^2 de 0 (o 1, dependiendo de como se considere)

In [35]:
from sklearn.metrics import r2_score
def train_salary(X_train, X_test, y_train, y_test,pipeline):
    pipeline.fit(X_train, y_train)
    display(pipeline)
    # Make predictions using the fitted pipeline
    y_pred = pipeline.predict(X_test)
    print("R^2 del modelo: ",r2_score(y_test,y_pred))


In [121]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
salarios_df = pd.read_csv("salarios.csv")
salarios_df=salarios_df.drop("Unnamed: 0",axis=1).rename({"Player":"Name"},axis=1)

full_caracs = pd.merge(df_players,salarios_df,how="inner",on="Name")
display(full_caracs)

X_salary = full_caracs.drop(["Weekly Salary"],axis=1)
Y_salary = full_caracs["Weekly Salary"]
categorical_salary = X_salary.select_dtypes(include=['object']).columns  # or exclude=['number'] for older versions
numerical_salary = X_salary.select_dtypes(include=['number']).columns

transformers_salary = [('num',PolynomialFeatures(degree=1, include_bias=False), numerical_salary),('cat', OneHotEncoder(handle_unknown='ignore'),categorical_salary)]
preprocessor_salary=ColumnTransformer(transformers_salary)

X_train_salary, X_test_salary, y_train_salary, y_test_salary = train_test_split(X_salary, Y_salary, test_size=0.2, random_state=1)

salary_pipeline = Pipeline([('preprocessor', preprocessor_salary),('regressor', LinearRegression())])

train_salary(X_train_salary, X_test_salary, y_train_salary, y_test_salary,salary_pipeline)

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Weekly Salary
0,Cristiano Ronaldo,Portugal,1,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1248536.0
1,Lionel Messi,Argentina,1,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1538905.0
2,Neymar,Brazil,1,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,797726.0
3,Luis Suárez,Uruguay,1,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,508923.0
4,Manuel Neuer,Germany,1,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,326233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Phillip Menzel,Germany,0,Res,191,83,Right,18,Medium / Medium,3,...,40,10,25,5,10,12,8,15,5,2034.0
1862,Manuel Akanji,Switzerland,0,Sub,187,85,Right,21,Medium / Medium,2,...,66,51,38,25,18,26,27,40,31,54176.0
1863,Moritz Nicolas,Germany,0,Res,195,87,Right,19,Medium / Medium,2,...,34,16,22,9,10,17,14,17,11,2262.0
1864,Giacomo Satalino,Italy,0,Sub,188,70,Right,17,Medium / Medium,1,...,48,11,19,6,5,13,11,18,9,2827.0


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  PolynomialFeatures(degree=1,
                                                                     include_bias=False),
                                                  Index(['National_Position', 'Height', 'Weight', 'Age', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pa...,
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Name', 'Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object'))])),
                ('regressor', LinearRegression())])

R^2 del modelo:  0.365131313808969


**Respuesta**:

En realidad si deberia ser posible obtener mejores resultados, puesto que el dataframe que se usa no tiene mas de 2000 filas, lo cual en terminos de evaluación de modelos es bastante poco. El R^2 se podria mejorar si se utilizara otra forma de escalar los datos, como utilizar escaladores logaritmos para el target o codificar de otra forma los datos categóricos. Como se puede ver, el modelo tiene un R^2 de 0.365, el cual es superior al que piden por lo que cumple con los requisitos, sin embargo, bajo esta metrica, aun esta lejos de ser un buen modelo debido a que la estimación no es tan cercana a los datos. Cabe decir que el R^2 es una medida que te dice que tan cercano estan los datos predichos con respecto a la esperanza de lo que se desea estimar, y eso por si solo no es un buen indicador, se deben utilizar otras métricas en conjunto a esta para poder decir con seguridad si el modelo esta haciendo una buena regresion o no

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>